# Mixed-Integer Nonlinear Programming (MINLP)

## What is Mixed-Integer Nonlinear Programming?
**Mixed-Integer Nonlinear Programming (MINLP)** extends **Nonlinear Programming (NLP)** by allowing **integer or binary decision variables**, similar to how **Mixed-Integer Linear Programming (MILP)** extends **Linear Programming (LP)**.

In short:
- **NLP** -> Nonlinear equations with continuous variables
- **MINLP** -> Nonlinear equations **+** integer / binary logic

This allows us to model problems that involve both:
- **Nonlinear system behavior** (e.g., exponential costs, efficiency curves)
- **Discrete decisions** (e.g., on/off, choose A or B)

---

## Why MINLP is Needed
Many real-world problems involve logical decisions that affect nonlinear behavior.  
For example:
- Turning a machine **on/off** changes nonlinear energy consumption
- Choosing a technology activates a different nonlinear cost function

To connect **logic decisions (yes/no, on/off)** with **nonlinear equations**, MINLP models often use techniques such as:
- **Big-M constraints**
- Indicator constraints
- Disjunctive formulations

--- 

## Production Cost Optimization Problem

Lets head back to the production cost problem from the [previous notebook](05_Non_Linear_Programming.ipynb) and adjust it so it simulates MINLP.

**Things that would be modified:**
* **Model Structure:**
    * The model is extended with **time steps**, allowing machines to be turned on or off at each hour using binary decision variables.
* **Constraints:**
    * The production efficiency loss and cost increase depend on the number of consecutive hours a machine operates without rest, capturing the effect of overheating.

### Problem Description
A manufacturer has developed an empirical model of its production system and has determined that the daily production cost is given by:

$\text{cost} = 0.4x_1^2 - 5x_1 + x_2^2 - 6x_2 + 3e^{0.1x_1} + 5e^{0.15x_2} + 50$

where $x_1$ and $x_2$ are the decision variables representing the number of hours per day that Unit 1 and Unit 2 are operated, respectively. The objective is to determine the optimal operating hours for both units that minimize the total production cost. In the **MINLP extension**, $x_1$ and $x_2$ are replaced by time-indexed runtime variables $R_{i,t}$. $R_{i,t}$ represents the number of consecutive hours machine i has been operating up to time t without interruption.

The manufacturer must produce at least 20,000 units per day.  
Unit 1 is an older but durable machine that produces 2,700 units per hour and can operate for up to 18 hours per day.  
Unit 2 is a newer and faster machine that produces 3,600 units per hour but can operate for at most 9 hours per day.

If a machine runs continuously, a production efficiency loss of $ (1 - \frac{1}{1 + x}) $ is applied, where $x$ represents the number of consecutive operating hours. This captures the gradual degradation in productivity due to overheating.

Note:  
Due to the presence of non-linear cost and efficiency functions, combined with binary on/off decision variables, the problem is formulated as a **Mixed Integer Non-Linear Programming (MINLP)** problem.

---
### Mathematical Formulation

**Sets:**  
The optimization problem is evaluated over a discrete time horizon consisting of 24 time steps.

$T$ – Set of all time steps, where $t \in T = \{0, 1, \dots, 23\}$
  
**Parameters:**  
$p_1 = 2700$: the number of units produced by machine 1  
$p_2 = 3600$: the number of units produced by machine 2

**Decision Variables:**  

$Y_{1,t} \in \{0,1\}$ – Machine 1 state (on/off) at time $t \in T$  
$Y_{2,t} \in \{0,1\}$ – Machine 2 state (on/off) at time $t \in T$  
$R_{1,t} \geq 0 $ – Machine 1 consecutive runtime  
$R_{2,t} \geq 0 $ – Machine 2 consecutive runtime  

**Objective:**  
$ Min: \text{cost} = \sum_{t \in T} ((0.4R_{1,t}^2 - 5R_{1,t} + 3e^{0.1R_{1,t}}) \times Y_{1,t}+ (R_{2,t}^2 - 6R_{2,t} + 5e^{0.15R_{2,t}}) \times Y_{2,t}) + 50$

**Constraints:**

1. **Runtime Limit:**  
$ \sum_{t \in T} Y_{1,t} \le 18 $  
$ \sum_{t \in T} Y_{2,t} \le 9 $

2. **Production Constraints:**  
$
\sum_{t \in T} \big[p_1 Y_{1,t} \big(1 - \frac{1}{1 + R_{1,t}}\big) + p_2 Y_{2,t} \big(1 - \frac{1}{1 + R_{2,t}}\big)\big] \geq 20000
$

**Big-M Constraints**
1. **Runtime Accumulation**  

$
R_{i,t} =
\begin{cases}
R_{i,t-1} + 1 & \text{if } Y_{i,t} = 1 \\
0 & \text{if } Y_{i,t} = 0
\end{cases}
\quad \forall i \in {1,2}, t \in T
$  
Covert to Big M:  

Since $M$ is the largest possible number in the context of Big M constraints, so in hourly context 24 is the largest

Therefore:  
* $ M = 24 $
* $R_{i,t} \geq 0 $

Runtime Accumulation when machine is on:  
$R_{i,t} \geq R_{i,t-1} + 1 - M(1 - Y_{i,t})$  

Gradual runtime accumulation:  
$R_{i,t} \leq R_{i,t-1} + 1$

Runtime Reset when $Y_{i,t} == 0$:  
$R_{i,t} \leq MY_{i,t}$ 

---
**The Meaning of Each Symbol**  
$x \in X$ - means `for x in X`  
$ \forall $ - means `for all`

## Code

### Import Libraries

In [1]:
import pyomo.environ as pyomo
from pyomo.environ import value

### Initialize Model

In [2]:
model = pyomo.ConcreteModel()

### Create Variables

In [3]:
# Initialize Timesteps
model.T = pyomo.RangeSet(0, 23)

# Decision variables
model.y1 = pyomo.Var(model.T, domain=pyomo.Binary)
model.y2 = pyomo.Var(model.T, domain=pyomo.Binary)
model.R1 = pyomo.Var(model.T, domain=pyomo.NonNegativeReals, bounds=(0,18))
model.R2 = pyomo.Var(model.T, domain=pyomo.NonNegativeReals, bounds=(0,9))

# Parameters
model.p1 = pyomo.Param(initialize = 2700)
model.p2 = pyomo.Param(initialize = 3600)

# M
model.M = pyomo.Param(initialize = 24)

###  Objective Function

In [4]:
def obj_func(model):
    return sum(
        (0.4 * model.R1[t]**2 - 5 * model.R1[t] + 3 * pyomo.exp(0.1 * model.R1[t])) * model.y1[t]
        + (model.R2[t]**2 - 6 * model.R2[t] + 5 * pyomo.exp(0.15 * model.R2[t])) * model.y2[t]
        for t in model.T
    ) + 50

model.objective = pyomo.Objective(rule = obj_func,
                                  sense = pyomo.minimize
                                  )

### Constraints

In [5]:
#1. Runtime Limit
def y1_runtime_limit(model):
    return sum([model.y1[t] for t in model.T]) <= 18
model.y1_runtime_limit = pyomo.Constraint(rule = y1_runtime_limit)

def y2_runtime_limit(model):
    return sum([model.y2[t] for t in model.T]) <= 9
model.y2_runtime_limit = pyomo.Constraint(rule = y2_runtime_limit)

#2. Production Constraints
def production_constraint(model):
    return sum([model.p1 * model.y1[t] * (1 - 1/(1+model.R1[t]))
                + model.p2 * model.y2[t] * (1 - 1/(1+model.R2[t]))
                for t in model.T
                ]) >= 20000
model.production_constraint = pyomo.Constraint(rule = production_constraint)

#3. Big M Constraint
## Runtime Accumulation
def r1_runtime_acc(model, t):
    if t == model.T.first():
        return model.R1[t] == model.y1[t]
    return model.R1[t] >= model.R1[t-1] + 1 - model.M * (1 - model.y1[t])
model.r1_runtime_acc = pyomo.Constraint(model.T, rule=r1_runtime_acc)

def r2_runtime_acc(model, t):
    if t == model.T.first():
        return model.R2[t] == model.y2[t]
    return model.R2[t] >= model.R2[t-1] + 1 - model.M * (1 - model.y2[t])
model.r2_runtime_acc = pyomo.Constraint(model.T, rule=r2_runtime_acc)

## Runtime Gradual Accumulation
def r1_runtime_gradual_acc(model, t):
    if t == model.T.first():
        return pyomo.Constraint.Skip
    return model.R1[t] <= model.R1[t-1] + 1 + model.M * (1 - model.y1[t])
model.r1_runtime_gradual_acc = pyomo.Constraint(model.T, rule = r1_runtime_gradual_acc)

def r2_runtime_gradual_acc(model, t):
    if t == model.T.first():
        return pyomo.Constraint.Skip
    return model.R2[t] <= model.R2[t-1] + 1 + model.M * (1 - model.y2[t])
model.r2_runtime_gradual_acc = pyomo.Constraint(model.T, rule = r2_runtime_gradual_acc)

## Runtime Reset when y{i,t} == 0
def r1_runtime_reset(model, t):
    return model.R1[t] <= model.M * model.y1[t]
model.r1_runtime_reset = pyomo.Constraint(model.T, rule = r1_runtime_reset)

def r2_runtime_reset(model, t):
    return model.R2[t] <= model.M * model.y2[t]
model.r2_runtime_reset = pyomo.Constraint(model.T, rule = r2_runtime_reset)

### Create Solver

In [6]:
# For the solver we will use BONMIN since this is an MINLP problem.
solver = pyomo.SolverFactory("bonmin")

In [7]:
solver.options["max_cpu_time"] = 60   # 1 minutes for time limit to for every trial
solver.options["bonmin.algorithm"] = "B-BB"
solver.solve(model, tee=True)

Bonmin 1.8.9 using Cbc 2.10.12 and Ipopt 3.14.19
bonmin: max_cpu_time=60
bonmin.algorithm=B-BB
max_cpu_time=60
bonmin.algorithm=B-BB


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -133.67197      200 52.895309
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -133.60161       65 17.951206
NLP0014I             2         OPT -138.04517       19 2.992653
NLP0014I             3         OPT -137.97481       23 5.20613
NLP0014I             4

KeyboardInterrupt: 

### Results

In [ ]:
print(f"{'t':<5}{'M1_on':<8}{'R1':<10}{'M2_on':<8}{'R2':<10}")
print("-" * 40)

for t in model.T:
    print(
        f"{t:<5}"
        f"{int(value(model.y1[t])):<8}"
        f"{value(model.R1[t]):<10.2f}"
        f"{int(value(model.y2[t])):<8}"
        f"{value(model.R2[t]):<10.2f}"
    )


t    M1_on   R1        M2_on   R2        
----------------------------------------
ERROR: evaluating object as numeric value: y1[0]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized VarData object y1[0]


ValueError: No value for uninitialized VarData object y1[0]

Some of the constraints/rules might lead to an infeasable result. Therefore there is no output for this notebook. I will try to reformulate the problem so this can be solved by the solver.